In [1]:
import h2o
import pandas as pd

In [2]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O cluster uptime:,20 hours 54 mins
H2O cluster timezone:,Asia/Seoul
H2O data parsing timezone:,UTC
H2O cluster version:,3.20.0.5
H2O cluster version age:,13 days
H2O cluster name:,H2O_from_python_root_rp8ljb
H2O cluster total nodes:,1
H2O cluster free memory:,13.15 Gb
H2O cluster total cores:,12
H2O cluster allowed cores:,12
H2O cluster status:,"locked, healthy"


In [3]:
data_set = pd.read_csv('rawToRemake_seq_1days_2up_5step_output.csv')
data_set.head(5)

,date,product_no,month,day,w_day,season,price,quantity_0,clicks_0,quantity_5,quantity_4,quantity_3,quantity_2,quantity_1,clicks_5,clicks_4,clicks_3,clicks_2,clicks_1,output
0,2017-01-06 00:00:00,97616,1,6,4,1,13800,0,3,0,2,0,0,0,0,3,3,3,5,0
1,2017-01-07 00:00:00,97616,1,7,5,1,13800,0,1,2,0,0,0,0,3,3,3,5,3,0
2,2017-03-03 00:00:00,97616,3,3,4,2,13800,1,14,0,0,0,0,0,14,25,12,11,11,0
3,2017-03-04 00:00:00,97616,3,4,5,2,13800,0,13,0,0,0,0,1,25,12,11,11,14,0
4,2017-03-05 00:00:00,97616,3,5,6,2,13800,0,9,0,0,0,1,0,12,11,11,14,13,0


In [4]:
del data_set['date']
del data_set['product_no']
data_set.head(5)

,month,day,w_day,season,price,quantity_0,clicks_0,quantity_5,quantity_4,quantity_3,quantity_2,quantity_1,clicks_5,clicks_4,clicks_3,clicks_2,clicks_1,output
0,1,6,4,1,13800,0,3,0,2,0,0,0,0,3,3,3,5,0
1,1,7,5,1,13800,0,1,2,0,0,0,0,3,3,3,5,3,0
2,3,3,4,2,13800,1,14,0,0,0,0,0,14,25,12,11,11,0
3,3,4,5,2,13800,0,13,0,0,0,0,1,25,12,11,11,14,0
4,3,5,6,2,13800,0,9,0,0,0,1,0,12,11,11,14,13,0


In [5]:
data_from_df = h2o.H2OFrame(data_set)
data_from_df.head(5)

/home/notebook/jhko01/AIMD_PLATFORM/lib/python2.7/site-packages/h2o/utils/shared_utils.py:177: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  data = _handle_python_lists(python_obj.as_matrix().tolist(), -1)[1]


Parse progress: |█████████████████████████████████████████████████████████| 100%


month,day,w_day,season,price,quantity_0,clicks_0,quantity_5,quantity_4,quantity_3,quantity_2,quantity_1,clicks_5,clicks_4,clicks_3,clicks_2,clicks_1,output
1,6,4,1,13800,0,3,0,2,0,0,0,0,3,3,3,5,0
1,7,5,1,13800,0,1,2,0,0,0,0,3,3,3,5,3,0
3,3,4,2,13800,1,14,0,0,0,0,0,14,25,12,11,11,0
3,4,5,2,13800,0,13,0,0,0,0,1,25,12,11,11,14,0
3,5,6,2,13800,0,9,0,0,0,1,0,12,11,11,14,13,0


In [6]:
data_split = data_from_df.split_frame(ratios = [0.8], seed = 1234)

data_train = data_split[0]
data_test = data_split[1]

In [7]:
x = data_train.columns
y = 'output'
x.remove(y)

In [8]:
data_train[y] = data_train[y].asfactor()
data_test[y] = data_test[y].asfactor()

In [11]:
saved_model = h2o.load_model('/home/notebook/jhko01/h2o_ex/cafe24_h2o_model/DRF_0_AutoML_20180822_151404')

In [34]:
pred = saved_model.predict(data_test)

drf prediction progress: |████████████████████████████████████████████████| 100%


In [57]:
pred.shape
from pandas import DataFrame

In [73]:

pred[:1,2:3]


p1
0.0961896


In [75]:
aa = pred.get_frame_data

In [79]:
aaa = h2o.as_list(pred, use_pandas=False)

In [81]:
print(aaa[1])

['0', '0.903810420036316', '0.09618957996368405']


In [82]:
print(aaa[1][2])

0.09618957996368405
